# Module 4 - Working with **Titan Multimodal Embeddings**

---

This notebook demonstrate how to generate and use embeddings for images and text using Amazon Titan Multimodal Embedding Models. We'll walk through how to extract these embeddings and perform similarity search with a query, laying out a path for building intelligent search and recommendation applications.

---

### Introduction

Amazon Titan Multimodal Embedding Models provide a simple and scalable way to represent images and text as embeddings—dense numerical vectors that capture semantic meaning. These models are ideal for building intelligent systems where understanding the similarity between images, texts, or both is critical.

Some key features of the Amazon Titan Multimodal Embedding Models include:

- **Multi-modal input support** - Encode text, images, or a combination of both into the same semantic space.

- **Enterprise-ready** - Built-in mechanisms to help mitigate bias in search results, support for multiple embedding dimensions for optimizing latency/accuracy trade-offs, and strong privacy and data security guarantees.

- **Flexible deployment** - Available through real-time inference and asynchronous batch transform APIs, and easily integrated with vector databases such as **Amazon OpenSearch Service**.

These models are pre-trained on large and diverse datasets, making them powerful out-of-the-box. For more specialized applications, you can also customize the embeddings using your own data, without needing to annotate large volumes of training examples.

This module will guide you through using Amazon Titan’s multimodal embeddings to extract image and text embeddings, store them in an index, and build a simple semantic search demo. Let’s get started!

### Pre-requisites

Please make sure that you have enabled the following model access in _Amazon Bedrock Console_:
- `Amazon Titan Multimodal Embeddings G1` (model ID: `amazon.titan-embed-image-v1`)
- `Amazon Titan Image Generator G1 (V2)` (model ID: `amazon.titan-image-generator-v2:0`)

## 1. Setup

### 1.1 Install and import the required libraries

In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
# Standard library imports
import os
import re
import sys
import json
import base64
from io import BytesIO

# Other library imports
import boto3
import numpy as np
import seaborn as sns
from PIL import Image
from scipy.spatial.distance import cdist

# Print SDK versions
print(f"Python version: {sys.version.split()[0]}")
print(f"Boto3 SDK version: {boto3.__version__}")

Python version: 3.12.9
Boto3 SDK version: 1.38.26


In [4]:
# Init boto session
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

# Init Bedrock Runtime client
bedrock_client = boto3.client("bedrock-runtime", region_name)

print("AWS Region:", region_name)

AWS Region: us-east-1


## 2. Synthetic Dataset

### 2.1 Generating Textual Description of Dataset Items with LLM

We can leverage Amazon Bedrock Language Models to randomly generate 7 different products, each with 3 variants, using prompt:

```
Generate a list of 7 items description for an online e-commerce shop, each comes with 3 variants of color or type. All with separate full sentence description.
```

Note that when using different language models, the reponses might be different. For illustration purpose, suppose we get the below response.

In [5]:
response = 'Here is a list of 7 items with 3 variants each for an online e-commerce shop, with separate full sentence descriptions:\n\n1. T-shirt\n- A red cotton t-shirt with a crew neck and short sleeves. \n- A blue cotton t-shirt with a v-neck and short sleeves.\n- A black polyester t-shirt with a scoop neck and cap sleeves.\n\n2. Jeans\n- Classic blue relaxed fit denim jeans with a mid-rise waist. \n- Black skinny fit denim jeans with a high-rise waist and ripped details at the knees.  \n- Stonewash straight leg denim jeans with a standard waist and front pockets.\n\n3. Sneakers  \n- White leather low-top sneakers with an almond toe cap and thick rubber outsole.\n- Gray mesh high-top sneakers with neon green laces and a padded ankle collar. \n- Tan suede mid-top sneakers with a round toe and ivory rubber cupsole.  \n\n4. Backpack\n- A purple nylon backpack with padded shoulder straps, front zipper pocket and laptop sleeve.\n- A gray canvas backpack with brown leather trims, side water bottle pockets and drawstring top closure.  \n- A black leather backpack with multiple interior pockets, top carry handle and adjustable padded straps.\n\n5. Smartwatch\n- A silver stainless steel smartwatch with heart rate monitor, GPS tracker and sleep analysis.  \n- A space gray aluminum smartwatch with step counter, phone notifications and calendar syncing. \n- A rose gold smartwatch with activity tracking, music controls and customizable watch faces.  \n\n6. Coffee maker\n- A 12-cup programmable coffee maker in brushed steel with removable water tank and keep warm plate.  \n- A compact 5-cup single serve coffee maker in matt black with travel mug auto-dispensing feature.\n- A retro style stovetop percolator coffee pot in speckled enamel with stay-cool handle and glass knob lid.  \n\n7. Yoga mat \n- A teal 4mm thick yoga mat made of natural tree rubber with moisture-wicking microfiber top.\n- A purple 6mm thick yoga mat made of eco-friendly TPE material with integrated carrying strap. \n- A patterned 5mm thick yoga mat made of PVC-free material with towel cover included.'
print(response)

Here is a list of 7 items with 3 variants each for an online e-commerce shop, with separate full sentence descriptions:

1. T-shirt
- A red cotton t-shirt with a crew neck and short sleeves. 
- A blue cotton t-shirt with a v-neck and short sleeves.
- A black polyester t-shirt with a scoop neck and cap sleeves.

2. Jeans
- Classic blue relaxed fit denim jeans with a mid-rise waist. 
- Black skinny fit denim jeans with a high-rise waist and ripped details at the knees.  
- Stonewash straight leg denim jeans with a standard waist and front pockets.

3. Sneakers  
- White leather low-top sneakers with an almond toe cap and thick rubber outsole.
- Gray mesh high-top sneakers with neon green laces and a padded ankle collar. 
- Tan suede mid-top sneakers with a round toe and ivory rubber cupsole.  

4. Backpack
- A purple nylon backpack with padded shoulder straps, front zipper pocket and laptop sleeve.
- A gray canvas backpack with brown leather trims, side water bottle pockets and drawstrin

The following function converts the response to a list of descriptions. You may need to write your own function depending on the real response.

In [6]:
def extract_text(input_string):
    pattern = r"- (.*?)($|\n)"
    matches = re.findall(pattern, input_string)
    extracted_texts = [match[0] for match in matches]
    return extracted_texts

Convert the response to a list of product descriptions.

In [7]:
product_descriptions = extract_text(response)
product_descriptions

['A red cotton t-shirt with a crew neck and short sleeves. ',
 'A blue cotton t-shirt with a v-neck and short sleeves.',
 'A black polyester t-shirt with a scoop neck and cap sleeves.',
 'Classic blue relaxed fit denim jeans with a mid-rise waist. ',
 'Black skinny fit denim jeans with a high-rise waist and ripped details at the knees.  ',
 'Stonewash straight leg denim jeans with a standard waist and front pockets.',
 'White leather low-top sneakers with an almond toe cap and thick rubber outsole.',
 'Gray mesh high-top sneakers with neon green laces and a padded ankle collar. ',
 'Tan suede mid-top sneakers with a round toe and ivory rubber cupsole.  ',
 'A purple nylon backpack with padded shoulder straps, front zipper pocket and laptop sleeve.',
 'A gray canvas backpack with brown leather trims, side water bottle pockets and drawstring top closure.  ',
 'A black leather backpack with multiple interior pockets, top carry handle and adjustable padded straps.',
 'A silver stainless st

### 2.2 Generating Image Pairs for the Textual Descriptions

The following function calls bedrock to generated images using "amazon.titan-image-generator-v1" model.

In [8]:
def titan_generate_image(payload, num_image=2, cfg=10.0, seed=2024):

    body = json.dumps(
        {
            **payload,
            "imageGenerationConfig": {
                "numberOfImages": num_image,   # Number of images to be generated. Range: 1 to 5 
                "quality": "premium",          # Quality of generated images. Can be standard or premium.
                "height": 1024,                # Height of output image(s)
                "width": 1024,                 # Width of output image(s)
                "cfgScale": cfg,               # Scale for classifier-free guidance. Range: 1.0 (exclusive) to 10.0
                "seed": seed                   # The seed to use for re-producibility. Range: 0 to 214783647
            }
        }
    )

    response = bedrock_client.invoke_model(
        body=body, 
        modelId="amazon.titan-image-generator-v2:0",
        accept="application/json", 
        contentType="application/json"
    )

    response_body = json.loads(response.get("body").read())
    images = [
        Image.open(
            BytesIO(base64.b64decode(base64_image))
        ) for base64_image in response_body.get("images")
    ]

    return images

Then we leverage the Titan Image Generation models to create product images for each of the descriptions. The following cell may take a few minutes to run.

In [9]:
embed_dir = "data/titan-embed"
os.makedirs(embed_dir, exist_ok=True)

titles = []
for i, prompt in enumerate(product_descriptions, 1):
    images = titan_generate_image(
        {
            "taskType": "TEXT_IMAGE",
            "textToImageParams": {
                "text": prompt, # Required
            }
        },
        num_image=1
    )
    title = "_".join(prompt.split()[:4]).lower()
    title = f"{embed_dir}/{title}.png"
    titles.append(title)
    images[0].save(title, format="png")
    print(f"[{i}/{len(product_descriptions)}] Generated: '{title}'..")

[1/21] Generated: 'data/titan-embed/a_red_cotton_t-shirt.png'..
[2/21] Generated: 'data/titan-embed/a_blue_cotton_t-shirt.png'..
[3/21] Generated: 'data/titan-embed/a_black_polyester_t-shirt.png'..
[4/21] Generated: 'data/titan-embed/classic_blue_relaxed_fit.png'..
[5/21] Generated: 'data/titan-embed/black_skinny_fit_denim.png'..
[6/21] Generated: 'data/titan-embed/stonewash_straight_leg_denim.png'..
[7/21] Generated: 'data/titan-embed/white_leather_low-top_sneakers.png'..
[8/21] Generated: 'data/titan-embed/gray_mesh_high-top_sneakers.png'..
[9/21] Generated: 'data/titan-embed/tan_suede_mid-top_sneakers.png'..
[10/21] Generated: 'data/titan-embed/a_purple_nylon_backpack.png'..
[11/21] Generated: 'data/titan-embed/a_gray_canvas_backpack.png'..
[12/21] Generated: 'data/titan-embed/a_black_leather_backpack.png'..
[13/21] Generated: 'data/titan-embed/a_silver_stainless_steel.png'..
[14/21] Generated: 'data/titan-embed/a_space_gray_aluminum.png'..
[15/21] Generated: 'data/titan-embed/a_ros

## 3. Multimodal Dataset Indexing

### 3.1 Embedding Images with Titan Multimodal Embeddings

The following function converts image, and optionally, text, into multimodal embeddings.

In [10]:
def titan_multimodal_embedding(
    image_path=None,  # maximum 2048 x 2048 pixels
    description=None, # English only and max input tokens 128
    dimension=1024,   # 1024 (default), 384, 256
    model_id="amazon.titan-embed-image-v1"
):
    payload_body = {}
    embedding_config = {
        "embeddingConfig": { 
             "outputEmbeddingLength": dimension
         }
    }

    # You can specify either text or image or both
    if image_path:
        with open(image_path, "rb") as image_file:
            input_image = base64.b64encode(image_file.read()).decode('utf8')
        payload_body["inputImage"] = input_image
    if description:
        payload_body["inputText"] = description

    assert payload_body, "please provide either an image and/or a text description"
    print("\n".join(payload_body.keys()))

    response = bedrock_client.invoke_model(
        body=json.dumps({**payload_body, **embedding_config}), 
        modelId=model_id,
        accept="application/json", 
        contentType="application/json"
    )

    return json.loads(response.get("body").read())

Now we can create embeddings for the generated images:

In [11]:
multimodal_embeddings = []
for title in titles:
    embedding = titan_multimodal_embedding(image_path=title, dimension=1024)["embedding"]
    multimodal_embeddings.append(embedding)
    print(f"generated embedding for {title}")

inputImage
generated embedding for data/titan-embed/a_red_cotton_t-shirt.png
inputImage
generated embedding for data/titan-embed/a_blue_cotton_t-shirt.png
inputImage
generated embedding for data/titan-embed/a_black_polyester_t-shirt.png
inputImage
generated embedding for data/titan-embed/classic_blue_relaxed_fit.png
inputImage
generated embedding for data/titan-embed/black_skinny_fit_denim.png
inputImage
generated embedding for data/titan-embed/stonewash_straight_leg_denim.png
inputImage
generated embedding for data/titan-embed/white_leather_low-top_sneakers.png
inputImage
generated embedding for data/titan-embed/gray_mesh_high-top_sneakers.png
inputImage
generated embedding for data/titan-embed/tan_suede_mid-top_sneakers.png
inputImage
generated embedding for data/titan-embed/a_purple_nylon_backpack.png
inputImage
generated embedding for data/titan-embed/a_gray_canvas_backpack.png
inputImage
generated embedding for data/titan-embed/a_black_leather_backpack.png
inputImage
generated emb

### 3.2 Analyze the Generated Image Embeddings

Let's see what we have generated so far:

In [12]:
print("Number of generated embeddings for images:", len(multimodal_embeddings))
print("Dimension of each image embedding:", len(multimodal_embeddings[-1]))
print("Example of generated embedding:\n", np.array(multimodal_embeddings[-1]))

Number of generated embeddings for images: 21
Dimension of each image embedding: 1024
Example of generated embedding:
 [-0.00793833  0.0202314  -0.02240877 ...  0.00578364 -0.00557951
 -0.00351555]


The following function produces a heatmap to display the inner product of the embeddings.

In [13]:
def plot_similarity_heatmap(embeddings_a, embeddings_b):
    inner_product = np.inner(embeddings_a, embeddings_b)
    sns.set(font_scale=1.1)
    graph = sns.heatmap(
        inner_product,
        vmin=np.min(inner_product),
        vmax=1,
        cmap="OrRd",
    )

Generate a heatmap to display the inner product of the embeddings. You can see that the diagonal is dark red, which means one embedding is closely related to itself. Then you can notice that there are 3X3 squares which are lighter than the diagonal, but darker than the rest. It means those 3 embeddings are less closely related to each other, than to itself, but more closely related to the rest embeddings. This makes sense, as we generated 3 variants of each product. Products are more closely related if they are of the same type. Products are less closely related if they are of different types.

In [ ]:
plot_similarity_heatmap(multimodal_embeddings, multimodal_embeddings)

## 4. Multimodal Search

We can now showcase a basic functionality of a multimodal search engine.

The following function returns the top similar multimodal embeddings given a query multimodal embedding. Note in practise you can leverage managed vector database, e.g. Amazon OpenSearch Service, and here is for illustration purpose.

In [ ]:
def search(query_emb:np.array, indexes:np.array, top_k:int=1):
    dist = cdist(query_emb, indexes, metric="cosine")
    return dist.argsort(axis=-1)[0,:top_k], np.sort(dist, axis=-1)[:top_k]

Now we have created the embeddings, we can search the list with a query, to find the product which the query best describes.

In [ ]:
query_prompt = "suede sneaker"
query_emb = titan_multimodal_embedding(description=query_prompt, dimension=1024)["embedding"]
len(query_emb)

In [ ]:
idx_returned, dist = search(
    np.array(query_emb)[None], 
    np.array(multimodal_embeddings)
)
idx_returned, dist

In [ ]:
for idx in idx_returned[:1]:
    display(Image.open(f"{titles[idx]}"))

Let's convert the above cells to a helper function.

In [ ]:
def multimodal_search(description:str, dimension:int):
    query_emb = titan_multimodal_embedding(description=description, dimension=dimension)["embedding"]

    idx_returned, dist = search(
        np.array(query_emb)[None], 
        np.array(multimodal_embeddings)
    )

    for idx in idx_returned[:1]:
        display(Image.open(f"{titles[idx]}"))

In [ ]:
multimodal_search(description="white sneaker", dimension=1024)

In [ ]:
multimodal_search(description="mesh sneaker", dimension=1024)

In [ ]:
multimodal_search(description="leather backpack", dimension=1024)

In [ ]:
multimodal_search(description="nylon backpack", dimension=1024)

In [ ]:
multimodal_search(description="canvas backpack", dimension=1024)

In [ ]:
multimodal_search(description="running shoes", dimension=1024)

## 5. Conclusions and Next Steps

In this module, we explored how to work with multimodal embeddings using Amazon Titan models. By embedding images and text into a shared semantic space, we demonstrated how to build powerful similarity search capabilities that go beyond traditional keyword matching. This approach opens up a wide range of possibilities for intelligent search, recommendation, and classification systems across industries such as e-commerce, media, and enterprise content management.

### Next Steps

Now please go on to explore the powerful capabilities of the Amazon Nova Canvas model to create compelling visual imagery for use cases such as product prototyping, dynamic content generation, and marketing asset creation:

&nbsp; **NEXT ▶** [2_nova-canvas-lab.ipynb](./2\_nova-canvas-lab.ipynb).